In [ ]:
import pandas as pd
#2) 판다스 데이터프레임 df_pd 생성
df_pd = pd.read_csv("projectdataO.csv", header = None, encoding = 'cp949')
df_pd.head()

#3) 파일을 하둡에서 읽어 SPDF 라는 spark 데이터프레임을 생성
from pyspark.sql import SparkSession
스파크 =  SparkSession.builder.appName('Test').getOrCreate()
스파크.conf.set("spark.sql.execution.arrow.enabled","true")
SPDF = 스파크.read.option('encoding', 'cp949').option('header','true').csv("hdfs://localhost:9000/Spark/spark_Employee.csv")

#4)df_pd 변수명을 변경
df_pd.columns = ['X1','X2', 'X3','X4','X5', 'X6','X7','X8', 'X9','X10']

#5) df_pd자료의 type을 조정하여 데이터 메모리 최소화
df_pd["X1"] = df_pd["X1"].astype('float32')
df_pd["X2"] = df_pd["X2"].astype('float32')
df_pd["X3"] = df_pd["X3"].astype('int8')
df_pd["X4"] = df_pd["X4"].astype('float32')
df_pd["X5"] = df_pd["X5"].astype('float32')
df_pd["X6"] = df_pd["X6"].astype('float32')
df_pd["X7"] = df_pd["X7"].astype('int32')
df_pd["X8"] = df_pd["X8"].astype('float32')
df_pd["X9"] = df_pd["X9"].astype('int64')
df_pd["X10"] = df_pd["X10"].astype('category')
df_pd.info()

#6) df_pd를 df_sp라는 스파크 데이터프레임으로 생성
df_sp =스파크.createDataFrame(df_pd)

#7) df_sp에 결측값이 포함된 행 제거
df_sp.na.drop().show()

#8) df_sp변수명이 X10의 범주별로 X9 를 기준으로 자료를 정렬
df_sp.orderBy(['X10',"X9"],ascending=False).show()

#9) df_sp자료를 하둡 에 parquet 파일로 저장
df_sp.write.parquet("hdfs://localhost:9000/SmallProject/projectO.parquet")

#10)df_pd에 포함되어 있는 수치자료를 자동확인하고 수치자료의 컬럼명을 df_Nname 대입
df_Nname = df_pd.select_dtypes(include=np.number).columns.tolist()

#11)df_pd의 각컬럼별로 결측값 개수 계산
df_pd.isnull().sum()

#12) df_pd의 X3이 50보다 크고 X8이 2 미만인 자료만 선택해서 df_sub에 저장
df_sub = df_pd.loc[((df_pd['X3'] >50) & (df_pd['X8'] >2))]

#13) df_pd X4 결측값을 나머지 수치자료들의 5NN 이용하여 대체
from sklearn.impute import KNNImputer
근접대체 = KNNImputer(n_neighbors=5, weights="uniform") #근처에 있는 값들을 정해 군집 분석 -> knn , k개의 값들의 평균 값을 대입
근접대체.set_output(transform='pandas') #5개를 기준으로 값들을 대체함
근접대체.fit_transform(pd.DataFrame(df_Nname['X4'])).head(20)

#14) df_pd의 X5 겨륵값을 X10의 범주별 평균으로 대체
df_pd['X5'].fillna(df_pd.groupby('X10')['X5'].mean())

#15) df_pd의 X6의 값을 4등분 하고 라벨 변경
df_pd['P'] = pd.qcut(df_pd['X6'], q = 4, labels=['C1', 'C2', 'C3', 'C4'])
df_pd['P']= df_pd[['P']].astype('category')

#16) df_pd의 X9를 X10의 범주별로 나누어 표준화하고 S에 대입
df_pd['S'] = df_pd.groupby(['X10'],group_keys=False)['X9'].apply(lambda x: (x-x.mean())/x.std())

#17) df_pd의 X10의 범주를 가변수로 만들어 저장
df_dummy = pd.get_dummies(df_pd['X10'])

#18) df_pd와 df_dummy를 같은 index끼리 결합하여 좌우 결합
df = pd.merge(df_pd, df_dummy,left_index=True, right_index=True)

#19) df_pd의 X10과 P 범주별로 교차하여 X1과 X2의 평균과 표준편차 구함
print(df_pd.groupby(['X10','P'])['X1'].agg(lambda x: x.std()))
print("\n")
print(df_pd.groupby(['X10','P'])['X1'].agg(lambda x: x.mean()))

print(df_pd.groupby(['X10','P'])['X2'].agg(lambda x: x.std()))
print("\n")
print(df_pd.groupby(['X10','P'])['X2'].agg(lambda x: x.mean()))

#20) df_pd의 칼럼 X1은 경도, X2는 위도 자료이며 이를 이용하여 지도 위에 위치를 표시하되X6의 값으로 원의 크기를  조절하고 X9의 값을 색깔로 표시
import matplotlib.pyplot as plt
df_pd.plot(kind='scatter', x='X1', y='X2', alpha=0.4, s=df_pd['X6']/30  ,
figsize=(12,8), )
plt.show()

###범주형 변수 -> 수치형 변수
from sklearn.preprocessing import OneHotEncoder
가변수 = OneHotEncoder()  
가변수_col = 가변수.fit_transform(df[['ocean_proximity']])가변수_col.toarray()
가변수DF = pd.DataFrame(가변수_col.toarray(),columns=["I1","I2","I3","I4","I5"],index=df.index)
new_df = pd.concat([df.iloc[:,0:-1],가변수DF],axis=1)
##순서자료를 수치자료로 변환
순서변환 = {'A':1,'B':2,'C':3,'D':4,'E':5}
순서구간 = 순서구간.map(순서변환).astype('uint8')